<a href="https://colab.research.google.com/github/kr5red/automated-customer-reviews/blob/main/Model3_Ki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IGNORE EVERYTHING BELOW UNTIL "START HERE"

In [13]:
from google.colab import files
uploaded = files.upload()

Saving best_products.pkl to best_products (1).pkl


In [14]:
from google.colab import files
uploaded = files.upload()

Saving worst_products.pkl to worst_products (1).pkl


In [15]:
!ls

'best_products (1).pkl'   sample_data		    worst_products.pkl
 best_products.pkl	 'worst_products (1).pkl'


In [16]:
import pickle

# Load the "best products" data/model
with open("best_products.pkl", "rb") as f:
    best_products = pickle.load(f)

# Load the "worst products" data/model
with open("worst_products.pkl", "rb") as f:
    worst_products = pickle.load(f)

In [17]:
print(type(best_products))
print(type(worst_products))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [18]:
import pandas as pd

print(best_products.head())

                       category_id predicted_product_category  \
0   Batteries_AV1YE_muvKc47QAVgpwE                  Batteries   
1   Batteries_AVpgNzjwLJeJML43Kpxn                  Batteries   
2   Batteries_AVpe7xlELJeJML43ypLz                  Batteries   
3   E-readers_AV1YnRtnglJLPUi8IJmV                  E-readers   
4   E-readers_AVpfl8cLLJeJML43AE3S                  E-readers   

                     id  count_new_id     brand_value  \
0  AV1YE_muvKc47QAVgpwE            40  Amazon Fire Tv   
1  AVpgNzjwLJeJML43Kpxn          8342    Amazonbasics   
2  AVpe7xlELJeJML43ypLz          3727    Amazonbasics   
3  AV1YnRtnglJLPUi8IJmV          3175          Amazon   
4  AVpfl8cLLJeJML43AE3S            45          Amazon   

                                        longest_name  \
0                                               None   
1  AmazonBasics AAA Performance Alkaline Batterie...   
2  AmazonBasics AA Performance Alkaline Batteries...   
3  Kindle Voyage E-reader, 6 High-Resoluti

In [19]:
print(worst_products.head())

                              category_id predicted_product_category  \
0          Batteries_AVpfIfGA1cnluZ0-emyp                  Batteries   
1          E-readers_AVpfiBlyLJeJML43-4Tp                  E-readers   
2          E-readers_AVpf_4sUilAPnD_xlwYV                  E-readers   
3          E-readers_AVpf_znpilAPnD_xlvAF                  E-readers   
4   Kids Electronics_AVpfIfGA1cnluZ0-emyp           Kids Electronics   

                     id  count_new_id                   brand_value  \
0  AVpfIfGA1cnluZ0-emyp            19                        Amazon   
1  AVpfiBlyLJeJML43-4Tp            11                        Amazon   
2  AVpf_4sUilAPnD_xlwYV            11                        Amazon   
3  AVpf_znpilAPnD_xlvAF            10  Amazon Digital Services Inc.   
4  AVpfIfGA1cnluZ0-emyp            11                        Amazon   

  longest_name                             summary_reviews_string  
0         None  Name: nan Title: works great Review: just boug...  
1   

## START HERE

## 1. Imports



In [20]:
!pip -q install mistralai tqdm

import os
import pickle
import math
from typing import List
from time import sleep

import pandas as pd
from tqdm.auto import tqdm
from mistralai import Mistral

## 3. Add API KEY (using mistralai SDK not Hugging face)



In [30]:
import os
from mistralai import Mistral

os.environ["MISTRAL_API_KEY"] = input("Enter your Mistral API key: ").strip()
MODEL_NAME = "mistral-small-latest"

client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])
print("Mistral client ready!")

KeyboardInterrupt: Interrupted by user

## 4. Define functions to call the Mistral model via the API

In [22]:
SYSTEM_PROMPT = (
    "You are an expert analyst of e-commerce customer reviews. "
    "Write an objective, concise synthesis that captures overall sentiment, "
    "Be specific and faithful to the reviews."
    "Keep it short with maximum 70 words and easy to read."
)
# Conservative chunk size (characters)
CHUNK_CHAR_LEN = 4000
MAX_RETRIES = 5
MAX_TOKENS_SHORT = 160
SHORT_TEXT_THRESHOLD = 1800

import time  # make sure you have this import near the top

def _mistral_chat(prompt: str) -> str:
    """Call Mistral chat with retries/backoff and a small delay to avoid rate limits."""
    delay = 2
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = client.chat.complete(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.2,
            )

            # ✅ Add a short delay here — after a successful call
            time.sleep(1.5)  # Adjust to 1–3 seconds if you still hit 429s

            return resp.choices[0].message.content.strip()

        except Exception as e:
            # Retry with exponential backoff if the request fails
            if attempt == MAX_RETRIES:
                raise
            time.sleep(delay)
            delay = min(delay * 2, 30)  # increase wait time between retries


#split long review text into chunks
def _split_into_chunks(text: str, chunk_size: int = CHUNK_CHAR_LEN) -> List[str]:
    text = text or ""
    if len(text) <= chunk_size:
        return [text]
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

#summarization process
def summarize_long_reviews(review_blob: str) -> str:
    """
    Map-reduce summarization for a potentially long `summary_reviews_string`.
    1) Split reviews into manageable chunks.
    2) Summarize each chunk.
    3) Summarize the chunk summaries into a final synthesis.
    """
    if not isinstance(review_blob, str) or not review_blob.strip():
        return ""

    chunks = _split_into_chunks(review_blob)

    # Map step: summarize each chunk
    chunk_summaries = []
    for idx, chunk in enumerate(chunks, 1):
        prompt = (
            f"Summarize the following set of customer reviews (part {idx} of {len(chunks)}). "
            f"REVIEWS:\n{chunk}"
        )
        chunk_summaries.append(_mistral_chat(prompt))

    # Reduce step: summarize the summaries
    reduce_prompt = (
        "Combine the following partial summaries into a single concise, structured summary. "
        f"PARTIAL SUMMARIES:\n" + "\n\n---\n\n".join(chunk_summaries)
    )
    final_summary = _mistral_chat(reduce_prompt)
    return final_summary.strip()

## 5. Create the new column

In [23]:
def add_review_summary_column(df: pd.DataFrame,
                              source_col: str = "summary_reviews_string",
                              target_col: str = "reviews_summary") -> pd.DataFrame:

    # progress-friendly apply
    tqdm.pandas(desc=f"Summarizing → {target_col}")
    df[target_col] = df[source_col].progress_apply(summarize_long_reviews)
    return df

best_products = add_review_summary_column(best_products, "summary_reviews_string", "reviews_summary")
worst_products = add_review_summary_column(worst_products, "summary_reviews_string", "reviews_summary")

print("'reviews_summary' added")

Summarizing → reviews_summary:   0%|          | 0/18 [00:00<?, ?it/s]

Summarizing → reviews_summary:   0%|          | 0/16 [00:00<?, ?it/s]

'reviews_summary' added


## 6. Check outcome

In [24]:
best_products[['summary_reviews_string', 'reviews_summary']].head()

,summary_reviews_string,reviews_summary
0,Name: nan Title: Excellent Review: Great produ...,Customers strongly favor the box for its speed...
1,Name: AmazonBasics AAA Performance Alkaline Ba...,Customers have mixed feelings about these batt...
2,Name: AmazonBasics AA Performance Alkaline Bat...,**Synthesis:**\n\nAmazonBasics AA Alkaline Bat...
3,Name: Amazon Kindle Paperwhite - eBook reader ...,**Synthesis:**\n\nCustomers highly praise the ...
4,"Name: Kindle Dx Leather Cover, Black (fits 9.7...","Customers overwhelmingly praise Amazon's Echo,..."


In [25]:
worst_products[['summary_reviews_string', 'reviews_summary']].head()

,summary_reviews_string,reviews_summary
0,Name: nan Title: works great Review: just boug...,Customers generally appreciate this charger fo...
1,Name: nan Title: as with everyone else Review:...,**Kindle Charger:** Mixed reviews. Praised for...
2,"Name: nan Title: It's a fine charger, just be ...",**Synthesis:**\n\nAmazon Kindle charger review...
3,Name: nan Title: Very pleased with cover Revie...,**Synthesis:**\n\nCustomers praise the cover's...
4,"Name: nan Title: Great Review: Easy to use, yo...",Customers generally appreciate this charger fo...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>